In [14]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d

In [2]:
SPECTRUM_FILE_PATH = r"D:\kexian.kuo\workspace\projects\1. YL-13 Gen2 專案\python\main\lib_from_customer\output (1) - 複製\25_blue_150_data.xlsx"

In [12]:
df = pd.read_excel(SPECTRUM_FILE_PATH)
df.to_numpy()[:, 0]

array(['校正前光譜強度', '校正後光譜波長', '校正後光譜強度', '亮度', '色座標', '波長'], dtype=object)

In [13]:
wavelengths = df.to_numpy()[0, 1:]
intensities = df.to_numpy()[0, 1:]

In [ ]:
# 加載 CIE 色彩匹配函數
cie_cmf = np.loadtxt("cie1931_cmf.csv", delimiter=",", skiprows=1)  # 波長, x̅, ȳ, z̅
cmf_wavelengths, x_bar, y_bar, z_bar = cie_cmf[:, 0], cie_cmf[:, 1], cie_cmf[:, 2], cie_cmf[:, 3]

# 插值強度數據
interp_func = interp1d(wavelengths, intensities, kind='linear', bounds_error=False, fill_value=0)
interp_intensities = interp_func(cmf_wavelengths)

# 計算三刺激值
k = 100 / np.trapezoid(interp_intensities * y_bar, cmf_wavelengths)
X = k * np.trapezoid(interp_intensities * x_bar, cmf_wavelengths)
Y = k * np.trapezoid(interp_intensities * y_bar, cmf_wavelengths)
Z = k * np.trapezoid(interp_intensities * z_bar, cmf_wavelengths)

# 計算 CIE 1931 色座標
if X + Y + Z != 0:
    x = X / (X + Y + Z)
    y = Y / (X + Y + Z)
else:
    x = y = 0

# 輸出結果
print(f"CIE 1931 色座標: x={x:.4f}, y={y:.4f}, 亮度 Y={Y:.2f}")